### IMPORTS

In [1]:
import torch
import numpy as np
import torchvision
from torch import nn
import sys
import os

import torchvision.transforms as transforms
import torchvision.datasets as datasets
import pandas as pd
import torch.optim as optim

from smithers.ml.vgg import VGG
from smithers.ml.utils import get_seq_model

import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore")


torch.cuda.empty_cache()
import datetime
import time

sys.path.insert(0, '../')
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')



batch_size = 8 #this can be changed
data_path = '../datasets/' 
# transform functions: take in input a PIL image and apply this
# transformations
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_dataset = datasets.CIFAR10(root=data_path + 'CIFAR10/',
                                 train=True,
                                 download=True,
                                 transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_dataset = datasets.CIFAR10(root=data_path + 'CIFAR10/',
                                train=False,
                                download=True,
                                transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
train_labels = torch.tensor(train_loader.dataset.targets).to(device)
targets = list(train_labels)



def save_checkpoint_torch(epoch, model, path, optimizer):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, path)


def load_checkpoint(model, checkpoint_path):
    model.load_state_dict(torch.load(checkpoint_path)['model_state_dict'])

Files already downloaded and verified
Files already downloaded and verified


### Accuracy della rete ridotta a partire dalla VGG trainata dal checkpoint e con red_model trainato con train_kd (red_method = 'POD')

In [2]:
start1 = time.time()


VGGcomplete = VGG(cfg=None,
                 classifier='cifar',
                 batch_norm=False,
                 num_classes=10,
                 init_weights=False,
                 pretrain_weights=None)
VGGcomplete = VGGcomplete.to(device)
VGGcomplete.make_layers()
VGGcomplete._initialize_weights()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(VGGcomplete.parameters(), lr=0.001, momentum=0.9)

pretrained = '/u/s/szanin/Smithers/smithers/ml/tutorials/check_vgg_cifar10_60_v2.pth.tar' #Stefano's
load_checkpoint(VGGcomplete, pretrained)
seq_model_complete = get_seq_model(VGGcomplete)
VGGcomplete = VGGcomplete.to(device)
seq_model_complete = seq_model_complete.to(device)

from smithers.ml.netadapter import NetAdapter

cutoff_idx = 7
red_dim = 50 
red_method = 'POD' 
inout_method = 'FNN'
n_class = 10

netadapter_complete = NetAdapter(cutoff_idx, red_dim, red_method, inout_method)
red_model_complete = netadapter_complete.reduce_net(seq_model_complete, train_dataset, train_labels, train_loader, n_class).to(device)
print(red_model_complete, flush=True)



mid1 = time.time()
print("L'inizializzazione ha impiegato {:.1f} minuti".format((mid1-start1)/60))
from smithers.ml.utils import Total_param, Total_flops
from smithers.ml.utils import compute_loss, train_kd



optimizer_complete = torch.optim.Adam([{
            'params': red_model_complete.premodel.parameters(),
            'lr': 1e-4
            }, {
            'params': red_model_complete.proj_model.parameters(),
            'lr': 1e-5
            }, {
            'params': red_model_complete.inout_map.parameters(),
            'lr': 1e-5
            }])

train_loss = []
test_loss = []
train_loss.append(compute_loss(red_model_complete, device, train_loader))
test_loss.append(compute_loss(red_model_complete, device, test_loader))

        
epochs = 10
filename = './cifar10_VGG16_RedNet_complete'+\
            '_cutIDx_%d.pth'%(cutoff_idx)

for epoch in range(1, epochs + 1):                       #da qui alla fine era dentro l'else commentato
    print('EPOCH {}'.format(epoch), flush=True)
    train_loss.append(
            train_kd(red_model_complete,
            VGGcomplete,
            device,
            train_loader,
            optimizer_complete,
            train_max_batch=200,
            alpha=0.1,
            temperature=1.,
            epoch=epoch))
    test_loss.append(compute_loss(red_model_complete, device, test_loader))
torch.save([red_model_complete.state_dict(), train_loss, test_loss], filename)
end1 = time.time()

total = 0
correct = 0
count = 0
for test, y_test in iter(test_loader):
    with torch.no_grad():
        output = red_model_complete(test)
        ps = torch.exp(output)
        _, predicted = torch.max(output.data,1)
        total += y_test.size(0)
        correct += (predicted == y_test.to(device)).sum().item() #MODIF
        count += 1
print('Accuracy of network on test images is {:.4f}....count:{}'.format(100*correct/total, count), flush=True)


time_VGGcomplete = end1 - start1
print('This took {:.1f} minutes'.format(time_VGGcomplete/60), flush=True)


Loaded base model.

Initializing reduction. Chosen reduction method is: POD
Le dimensioni delle due matrici sono: proj_mat = torch.Size([4096, 50]) e matrix = torch.Size([50000, 4096])
Epoch 0 of 500


















































Epoch 50 of 500


















































Epoch 100 of 500


















































Epoch 150 of 500


















































Epoch 200 of 500


















































Epoch 250 of 500


















































Epoch 300 of 500


















































Epoch 350 of 500


















































Epoch 400 of 500


















































Epoch 450 of 500


















































RedNet(
  (premodel): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Co

### Accuracy della rete ridotta a partire dalla VGG trainata dal checkpoint e con red_model trainato con train_kd (red_method = 'RandSVD)

In [2]:
start1 = time.time()

VGGcomplete = VGG(cfg=None,
                 classifier='cifar',
                 batch_norm=False,
                 num_classes=10,
                 init_weights=False,
                 pretrain_weights=None)
VGGcomplete = VGGcomplete.to(device)
VGGcomplete.make_layers()
VGGcomplete._initialize_weights()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(VGGcomplete.parameters(), lr=0.001, momentum=0.9)

pretrained = '/u/s/szanin/Smithers/smithers/ml/tutorials/check_vgg_cifar10_60_v2.pth.tar' #Stefano's
load_checkpoint(VGGcomplete, pretrained)
seq_model_complete = get_seq_model(VGGcomplete)
VGGcomplete = VGGcomplete.to(device)
seq_model_complete = seq_model_complete.to(device)

from smithers.ml.netadapter import NetAdapter

cutoff_idx = 7
red_dim = 50 
red_method = 'RandSVD' 
inout_method = 'FNN'
n_class = 10

netadapter_complete = NetAdapter(cutoff_idx, red_dim, red_method, inout_method)
red_model_complete = netadapter_complete.reduce_net(seq_model_complete, train_dataset, train_labels, train_loader, n_class).to(device)
print(red_model_complete, flush=True)



mid1 = time.time()
print("L'inizializzazione ha impiegato {:.1f} minuti".format((mid1-start1)/60))
from smithers.ml.utils import Total_param, Total_flops
from smithers.ml.utils import compute_loss, train_kd



optimizer_complete = torch.optim.Adam([{
            'params': red_model_complete.premodel.parameters(),
            'lr': 1e-4
            }, {
            'params': red_model_complete.proj_model.parameters(),
            'lr': 1e-5
            }, {
            'params': red_model_complete.inout_map.parameters(),
            'lr': 1e-5
            }])

train_loss = []
test_loss = []
train_loss.append(compute_loss(red_model_complete, device, train_loader))
test_loss.append(compute_loss(red_model_complete, device, test_loader))

        
epochs = 10
filename = './cifar10_VGG16_RedNet_complete_'+str(red_method)+\
            '_cutIDx_%d.pth'%(cutoff_idx)

for epoch in range(1, epochs + 1):                       #da qui alla fine era dentro l'else commentato
    print('EPOCH {}'.format(epoch), flush=True)
    train_loss.append(
            train_kd(red_model_complete,
            VGGcomplete,
            device,
            train_loader,
            optimizer_complete,
            train_max_batch=200,
            alpha=0.1,
            temperature=1.,
            epoch=epoch))
    test_loss.append(compute_loss(red_model_complete, device, test_loader))
torch.save([red_model_complete.state_dict(), train_loss, test_loss], filename)
end1 = time.time()

total = 0
correct = 0
count = 0
for test, y_test in iter(test_loader):
    with torch.no_grad():
        output = red_model_complete(test)
        ps = torch.exp(output)
        _, predicted = torch.max(output.data,1)
        total += y_test.size(0)
        correct += (predicted == y_test.to(device)).sum().item() #MODIF
        count += 1
print('Accuracy of network on test images is {:.4f}....count:{}'.format(100*correct/total, count), flush=True)


time_VGGcomplete = end1 - start1
print('This took {:.1f} minutes'.format(time_VGGcomplete/60), flush=True)


Loaded base model.

Initializing reduction. Chosen reduction method is: RandSVD
Siamo alla batch batch 0
Siamo alla batch batch 1000
Siamo alla batch batch 2000
Siamo alla batch batch 3000
Siamo alla batch batch 4000
Siamo alla batch batch 5000
Siamo alla batch batch 6000
Siamo alla batch batch 0
Siamo alla batch batch 1000
Siamo alla batch batch 2000
Siamo alla batch batch 3000
Siamo alla batch batch 4000
Siamo alla batch batch 5000
Siamo alla batch batch 6000
Le dimensioni delle due matrici sono: proj_mat = torch.Size([4096, 50]) e matrix (input di projection) = torch.Size([50000, 4096])
si dovranno moltiplicare alcune righe di input_matrix per proj_matrix
proj_mat è salvata su 0 (-1 = cpu, 0 = gpu)
matrix è salvata su 0 (-1 = cpu, 0 = gpu)
Comincia ora il training della FNN
Epoch 0 of 500
Epoch 50 of 500
Epoch 100 of 500
Epoch 150 of 500
Epoch 200 of 500
Epoch 250 of 500
Epoch 300 of 500
Epoch 350 of 500
Epoch 400 of 500
Epoch 450 of 500
È terminato il training della rete neurale
R

### Accuracy della rete ridotta a partire da VGG non trainata, train_kd per 60 epoche, teacher non trainato (red_method='RandSVD')

In [3]:
start3 = time.time()


VGGmix = VGG(cfg=None,
                 classifier='cifar',
                 batch_norm=False,
                 num_classes=10,
                 init_weights=False,
                 pretrain_weights=None)
VGGmix = VGGmix.to(device)
VGGmix.make_layers()
VGGmix._initialize_weights()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(VGGmix.parameters(), lr=0.001, momentum=0.9)


seq_model_mix = get_seq_model(VGGmix)
VGGmix = VGGmix.to(device)
seq_model_mix = seq_model_mix.to(device)

from smithers.ml.netadapter import NetAdapter

cutoff_idx = 7
red_dim = 50 
red_method = 'RandSVD' 
inout_method = 'FNN'
n_class = 10

netadapter_mix = NetAdapter(cutoff_idx, red_dim, red_method, inout_method)
red_model_mix = netadapter_mix.reduce_net(seq_model_mix, train_dataset, train_labels, train_loader, n_class).to(device)
print(red_model_mix, flush=True)




from smithers.ml.utils import Total_param, Total_flops
from smithers.ml.utils import compute_loss, train_kd

mid3 = time.time()
print('Tempo di inizializzazione: {:.1f} minuti'.format((mid3 -start3)/60), flush=True)



optimizer_mix = torch.optim.Adam([{
            'params': red_model_mix.premodel.parameters(),
            'lr': 1e-4
            }, {
            'params': red_model_mix.proj_model.parameters(),
            'lr': 1e-5
            }, {
            'params': red_model_mix.inout_map.parameters(),
            'lr': 1e-5
            }])

train_loss = []
test_loss = []
train_loss.append(compute_loss(red_model_mix, device, train_loader))
test_loss.append(compute_loss(red_model_mix, device, test_loader))

        
epochs = 60
filename = './cifar10_VGG16_RedNet_mix'+\
            '_cutIDx_%d.pth'%(cutoff_idx)

for epoch in range(1, epochs + 1):                       #da qui alla fine era dentro l'else commentato
    print('EPOCH {}'.format(epoch), flush=True)
    train_loss.append(
            train_kd(red_model_mix,
            VGGmix,
            device,
            train_loader,
            optimizer_mix,
            train_max_batch=200,
            alpha=0.1,
            temperature=1.,
            epoch=epoch))
    test_loss.append(compute_loss(red_model_mix, device, test_loader))
#torch.save([red_model_mix.state_dict(), train_loss, test_loss], filename)

end3 = time.time()
time_VGGmix = end3 - start3

total = 0
correct = 0
count = 0
for test, y_test in iter(test_loader):
    with torch.no_grad():
        output = red_model_mix(test)
        ps = torch.exp(output)
        _, predicted = torch.max(output.data,1)
        total += y_test.size(0)
        correct += (predicted == y_test.to(device)).sum().item() 
        count += 1
print('Accuracy of network on test images is {:.4f}....count:{}'.format(100*correct/total, count), flush=True)


print('Tempo totale {:.1f} minutes'.format(time_VGGmix/60), flush=True)


Loaded base model.

Initializing reduction. Chosen reduction method is: RandSVD
Siamo alla batch batch 0
Siamo alla batch batch 1000
Siamo alla batch batch 2000
Siamo alla batch batch 3000
Siamo alla batch batch 4000
Siamo alla batch batch 5000
Siamo alla batch batch 6000
Siamo alla batch batch 0
Siamo alla batch batch 1000
Siamo alla batch batch 2000
Siamo alla batch batch 3000
Siamo alla batch batch 4000
Siamo alla batch batch 5000
Siamo alla batch batch 6000
Le dimensioni delle due matrici sono: proj_mat = torch.Size([4096, 50]) e matrix (input di projection) = torch.Size([50000, 4096])
si dovranno moltiplicare alcune righe di input_matrix per proj_matrix
proj_mat è salvata su 0 (-1 = cpu, 0 = gpu)
matrix è salvata su 0 (-1 = cpu, 0 = gpu)
Comincia ora il training della FNN
Epoch 0 of 500
Epoch 50 of 500
Epoch 100 of 500
Epoch 150 of 500
Epoch 200 of 500
Epoch 250 of 500
Epoch 300 of 500
Epoch 350 of 500
Epoch 400 of 500
Epoch 450 of 500
È terminato il training della rete neurale
R

### Accuracy della rete ridotta a partire da una VGG non trainata e con red_model trainato per 60 epoche rispetto un teacher trainato (red_method = 'POD')

In [ ]:
start4 = time.time()


VGGmixtrained = VGG(cfg=None,
                 classifier='cifar',
                 batch_norm=False,
                 num_classes=10,
                 init_weights=False,
                 pretrain_weights=None)
VGGmixtrained = VGGmixtrained.to(device)
VGGmixtrained.make_layers()
VGGmixtrained._initialize_weights()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(VGGmixtrained.parameters(), lr=0.001, momentum=0.9)


seq_model_mixtrained = get_seq_model(VGGmixtrained)
VGGmixtrained = VGGmixtrained.to(device)
seq_model_mixtrained = seq_model_mixtrained.to(device)

from smithers.ml.netadapter import NetAdapter

cutoff_idx = 7
red_dim = 50 
red_method = 'POD' 
inout_method = 'FNN'
n_class = 10

netadapter_mixtrained = NetAdapter(cutoff_idx, red_dim, red_method, inout_method)
red_model_mixtrained = netadapter_mixtrained.reduce_net(seq_model_mixtrained, train_dataset, train_labels, train_loader, n_class).to(device)
print(red_model_mixtrained, flush=True)


mid4 = time.time()
print('Tempo di inizializzazione: {} minuti'.format((mid4 -start4)/60), flush=True)

from smithers.ml.utils import Total_param, Total_flops
from smithers.ml.utils import compute_loss, train_kd


optimizer_mixtrained = torch.optim.Adam([{
            'params': red_model_mixtrained.premodel.parameters(),
            'lr': 1e-4
            }, {
            'params': red_model_mixtrained.proj_model.parameters(),
            'lr': 1e-5
            }, {
            'params': red_model_mixtrained.inout_map.parameters(),
            'lr': 1e-5
            }])

train_loss = []
test_loss = []
train_loss.append(compute_loss(red_model_mixtrained, device, train_loader))
test_loss.append(compute_loss(red_model_mixtrained, device, test_loader))

        
epochs = 60
filename = './cifar10_VGG16_RedNet_mixtrained'+\
            '_cutIDx_%d.pth'%(cutoff_idx)

VGGteacher = VGG(cfg=None,
                 classifier='cifar',
                 batch_norm=False,
                 num_classes=10,
                 init_weights=False,
                 pretrain_weights=None)
VGGteacher = VGGteacher.to(device)
VGGteacher.make_layers()
VGGteacher._initialize_weights()
criterion_teacher = nn.CrossEntropyLoss()
optimizer_teacher = optim.SGD(VGGteacher.parameters(), lr=0.001, momentum=0.9)

pretrained = '/u/s/szanin/Smithers/smithers/ml/tutorials/check_vgg_cifar10_60_v2.pth.tar' #Stefano's
load_checkpoint(VGGteacher, pretrained)
seq_model_teacher = get_seq_model(VGGteacher)
VGGteacher = VGGteacher.to(device)
seq_model_teacher = seq_model_teacher.to(device)


for epoch in range(1, epochs + 1):                       #da qui alla fine era dentro l'else commentato
    print('EPOCH {}'.format(epoch), flush=True)
    train_loss.append(
            train_kd(red_model_mixtrained,
            VGGteacher,
            device,
            train_loader,
            optimizer_mixtrained,
            train_max_batch=200,
            alpha=0.1,
            temperature=1.,
            epoch=epoch))
    test_loss.append(compute_loss(red_model_mixtrained, device, test_loader))
#torch.save([red_model_mixtrained.state_dict(), train_loss, test_loss], filename)
end4 = time.time()

total = 0
correct = 0
count = 0
for test, y_test in iter(test_loader):
    with torch.no_grad():
        output = red_model_mixtrained(test)
        ps = torch.exp(output)
        _, predicted = torch.max(output.data,1)
        total += y_test.size(0)
        correct += (predicted == y_test.to(device)).sum().item() 
        count += 1
print('Accuracy of network on test images is {:.4f}....count:{}'.format(100*correct/total, count), flush=True)

time_VGGmixtrained = end4 - start4
print(time_VGGmixtrained)

### Accuracy della rete ridotta a partire da una VGG non trainata e con red_model trainato per 60 epoche rispetto un teacher trainato (red_method = 'RandSVD')

In [ ]:
start4 = time.time()


VGGmixtrained = VGG(cfg=None,
                 classifier='cifar',
                 batch_norm=False,
                 num_classes=10,
                 init_weights=False,
                 pretrain_weights=None)
VGGmixtrained = VGGmixtrained.to(device)
VGGmixtrained.make_layers()
VGGmixtrained._initialize_weights()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(VGGmixtrained.parameters(), lr=0.001, momentum=0.9)


seq_model_mixtrained = get_seq_model(VGGmixtrained)
VGGmixtrained = VGGmixtrained.to(device)
seq_model_mixtrained = seq_model_mixtrained.to(device)

from smithers.ml.netadapter import NetAdapter

cutoff_idx = 7
red_dim = 50 
red_method = 'RandSVD' 
inout_method = 'FNN'
n_class = 10

netadapter_mixtrained = NetAdapter(cutoff_idx, red_dim, red_method, inout_method)
red_model_mixtrained = netadapter_mixtrained.reduce_net(seq_model_mixtrained, train_dataset, train_labels, train_loader, n_class).to(device)
print(red_model_mixtrained, flush=True)


mid4 = time.time()
print('Tempo di inizializzazione: {} minuti'.format((mid4 -start4)/60), flush=True)

from smithers.ml.utils import Total_param, Total_flops
from smithers.ml.utils import compute_loss, train_kd


optimizer_mixtrained = torch.optim.Adam([{
            'params': red_model_mixtrained.premodel.parameters(),
            'lr': 1e-4
            }, {
            'params': red_model_mixtrained.proj_model.parameters(),
            'lr': 1e-5
            }, {
            'params': red_model_mixtrained.inout_map.parameters(),
            'lr': 1e-5
            }])

train_loss = []
test_loss = []
train_loss.append(compute_loss(red_model_mixtrained, device, train_loader))
test_loss.append(compute_loss(red_model_mixtrained, device, test_loader))

        
epochs = 60
filename = './cifar10_VGG16_RedNet_mixtrained'+\
            '_cutIDx_%d.pth'%(cutoff_idx)

VGGteacher = VGG(cfg=None,
                 classifier='cifar',
                 batch_norm=False,
                 num_classes=10,
                 init_weights=False,
                 pretrain_weights=None)
VGGteacher = VGGteacher.to(device)
VGGteacher.make_layers()
VGGteacher._initialize_weights()
criterion_teacher = nn.CrossEntropyLoss()
optimizer_teacher = optim.SGD(VGGteacher.parameters(), lr=0.001, momentum=0.9)

pretrained = '/u/s/szanin/Smithers/smithers/ml/tutorials/check_vgg_cifar10_60_v2.pth.tar' #Stefano's
load_checkpoint(VGGteacher, pretrained)
seq_model_teacher = get_seq_model(VGGteacher)
VGGteacher = VGGteacher.to(device)
seq_model_teacher = seq_model_teacher.to(device)


for epoch in range(1, epochs + 1):                       #da qui alla fine era dentro l'else commentato
    print('EPOCH {}'.format(epoch), flush=True)
    train_loss.append(
            train_kd(red_model_mixtrained,
            VGGteacher,
            device,
            train_loader,
            optimizer_mixtrained,
            train_max_batch=200,
            alpha=0.1,
            temperature=1.,
            epoch=epoch))
    test_loss.append(compute_loss(red_model_mixtrained, device, test_loader))
#torch.save([red_model_mixtrained.state_dict(), train_loss, test_loss], filename)
end4 = time.time()

total = 0
correct = 0
count = 0
for test, y_test in iter(test_loader):
    with torch.no_grad():
        output = red_model_mixtrained(test)
        ps = torch.exp(output)
        _, predicted = torch.max(output.data,1)
        total += y_test.size(0)
        correct += (predicted == y_test.to(device)).sum().item() 
        count += 1
print('Accuracy of network on test images is {:.4f}....count:{}'.format(100*correct/total, count), flush=True)

time_VGGmixtrained = end4 - start4
print(time_VGGmixtrained)